In [1]:
# packages
import pymysql.cursors
import pandas as pd
import os

In [2]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'Data',
 'Data Warehouse',
 'etl.py',
 'modelagem_dimensional.ipynb']

In [25]:
# reading just 100000 rows the dataframe 
df = pd.read_csv('Data/201701_BolsaFamiliaFolhaPagamento.csv', encoding = "ISO-8859-1", error_bad_lines=False, sep='\t', nrows=100000)

In [3]:
df.shape

(100000, 12)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
UF                        100000 non-null object
Código SIAFI Município    100000 non-null int64
Nome Município            100000 non-null object
Código Função             100000 non-null int64
Código Subfunção          100000 non-null int64
Código Programa           100000 non-null int64
Código Ação               100000 non-null int64
NIS Favorecido            100000 non-null int64
Nome Favorecido           100000 non-null object
Fonte-Finalidade          100000 non-null object
Valor Parcela             100000 non-null object
Mês Competência           100000 non-null object
dtypes: int64(6), object(6)
memory usage: 9.2+ MB


In [5]:
df.head(3)

,UF,Código SIAFI Município,Nome Município,Código Função,Código Subfunção,Código Programa,Código Ação,NIS Favorecido,Nome Favorecido,Fonte-Finalidade,Valor Parcela,Mês Competência
0,BA,3389,BROTAS DE MACAUBAS,8,244,1335,8442,16074176737,FLAVIANO SEBASTIAO DOS SANTOS,CAIXA - Programa Bolsa Família,131.00,01/2017
1,PE,2531,RECIFE,8,244,1335,8442,20033347012,ELANE PATRICIA DA SILVA DAMASIO NUNES,CAIXA - Programa Bolsa Família,170.00,01/2017
2,SP,6291,CAMPINAS,8,244,1335,8442,13287063897,ANGELA APARECIDA SERAFIM,CAIXA - Programa Bolsa Família,202.00,01/2017


In [27]:
df['Valor Parcela'] = df['Valor Parcela'].apply(lambda num: float(num.replace(',', '')) if ',' in num else float(num))


In [31]:
df[df['UF'] == 'RJ'].describe()

,Código SIAFI Município,Código Função,Código Subfunção,Código Programa,Código Ação,NIS Favorecido,Valor Parcela
count,5772.000000,5772.0,5772.0,5772.0,5772.0,5.772000e+03,5772.000000
mean,5462.788981,8.0,244.0,1335.0,8442.0,1.654944e+10,171.905405
std,1209.375625,0.0,0.0,0.0,0.0,3.701750e+09,98.021983
min,770.000000,8.0,244.0,1335.0,8442.0,1.006371e+10,39.000000
25%,5833.000000,8.0,244.0,1335.0,8442.0,1.294968e+10,87.000000
50%,5875.000000,8.0,244.0,1335.0,8442.0,1.636125e+10,163.000000
75%,6001.000000,8.0,244.0,1335.0,8442.0,2.040643e+10,209.000000
max,6009.000000,8.0,244.0,1335.0,8442.0,2.376529e+10,1362.000000


In [7]:
for col in df.columns:
    print("Coluna: ", col)

Coluna:  UF
Coluna:  Código SIAFI Município
Coluna:  Nome Município
Coluna:  Código Função
Coluna:  Código Subfunção
Coluna:  Código Programa
Coluna:  Código Ação
Coluna:  NIS Favorecido
Coluna:  Nome Favorecido
Coluna:  Fonte-Finalidade
Coluna:  Valor Parcela
Coluna:  Mês Competência


In [8]:
df['Regiao'] = None

In [9]:
# add region name to dataframe 
def regiao(estado):
    if estado in ['MA','PI','CE','RN','PE','PB','SE','AL','BA']:
        return 'Nordeste'
    elif estado in ['AM', 'RR', 'RO', 'AP', 'TO', 'RO', 'AC']:
        return 'Norte'
    elif estado in ['MT', 'MS', 'GO', 'DF']:
        return 'Centro-Oeste'
    elif estado in ['SP', 'RJ', 'ES', 'MG']:
        return 'Sudeste'
    else:
        return 'Sul'

In [10]:
df['Regiao'] = df['UF'].apply(regiao)

In [11]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Fonte-Finalidade', 'Valor Parcela',
       'Mês Competência', 'Regiao'],
      dtype='object')

In [14]:
# returning state's name to datafrane
def estado(uf):
    if uf == 'MA':
        return 'Maranhão'
    elif uf == 'PI':
        return 'Piauí'
    elif uf == 'CE':
        return 'Ceará'
    elif uf == 'BA':
        return 'Bahia'
    elif uf == 'AL':
        return 'Alagoas'
    elif uf == 'SE':
        return 'Sergipe'
    elif uf == 'RN':
        return 'Rio Grande do Norte'
    elif uf == 'PE':
        return 'Pernambuco'
    elif uf == 'PB':
        return 'Paraíba'
    elif uf == 'AM':
        return 'Amazonas'
    elif uf == 'RR':
        return 'Roraima'
    elif uf == 'AP':
        return 'Amapá'
    elif uf == 'PA':
        return 'Pará'
    elif uf == 'TO':
        return 'Tocantins'
    elif uf == 'RO':
        return 'Rondonia'
    elif uf == 'AC':
        return 'Acre'
    elif uf == 'MT':
        return 'Mato Grosso'
    elif uf == 'MS':
        return 'Mato Grosso do Sul'
    elif uf == 'GO':
        return 'Goiás'
    elif uf == 'SP':
        return 'São Paulo'
    elif uf == 'RJ':
        return 'Rio de Janeiro'
    elif uf == 'ES':
        return 'Espirito Santo'
    elif uf == 'MG':
        return 'Minas Gerais'
    elif uf == 'PR':
        return 'Paraná'
    elif uf == 'RS':
        return 'Rio Grande do Sul'
    elif uf == 'SC':
        return 'Santa Catarina'
    else:
        return ''

In [15]:
#df['Estado'] = None
df['Estado'] = df['UF'].apply(estado)

In [16]:
df['Fonte'] = df['Fonte-Finalidade'].apply(lambda x: x.split('-')[0])
df['Finalidade'] = df['Fonte-Finalidade'].apply(lambda x: x.split('-')[1])
df.drop('Fonte-Finalidade', axis=1, inplace = True)

In [18]:
df.head(3)

,UF,Código SIAFI Município,Nome Município,Código Função,Código Subfunção,Código Programa,Código Ação,NIS Favorecido,Nome Favorecido,Valor Parcela,Mês Competência,Regiao,Estado,Fonte,Finalidade
0,BA,3389,BROTAS DE MACAUBAS,8,244,1335,8442,16074176737,FLAVIANO SEBASTIAO DOS SANTOS,131.0,01/2017,Nordeste,Bahia,CAIXA,Programa Bolsa Família
1,PE,2531,RECIFE,8,244,1335,8442,20033347012,ELANE PATRICIA DA SILVA DAMASIO NUNES,170.0,01/2017,Nordeste,Pernambuco,CAIXA,Programa Bolsa Família
2,SP,6291,CAMPINAS,8,244,1335,8442,13287063897,ANGELA APARECIDA SERAFIM,202.0,01/2017,Sudeste,São Paulo,CAIXA,Programa Bolsa Família


In [19]:
df['Estado'].unique()
df['Regiao'].unique()

array(['Nordeste', 'Sudeste', 'Centro-Oeste', 'Sul', 'Norte'],
      dtype=object)

In [20]:
df[df['Nome Município'] == 'RECIFE']['Código SIAFI Município'].unique()[0]

2531

In [22]:
df.isnull().any()

UF                        False
Código SIAFI Município    False
Nome Município            False
Código Função             False
Código Subfunção          False
Código Programa           False
Código Ação               False
NIS Favorecido            False
Nome Favorecido           False
Valor Parcela             False
Mês Competência           False
Regiao                    False
Estado                    False
Fonte                     False
Finalidade                False
dtype: bool

In [24]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Valor Parcela', 'Mês Competência', 'Regiao',
       'Estado', 'Fonte', 'Finalidade'],
      dtype='object')

In [49]:
#df['Código Função'].unique()
#df['Código Ação'].unique()
df['Código Subfunção'].unique()

array([244], dtype=int64)

In [44]:
for data in df['Mês Competência'].unique():
    print(data.split('/'))

['01', '2017']
['12', '2016']
['11', '2016']


In [56]:
dataset_aux = df[df['Código Função'] == 8]
dataset_aux.info()             


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
UF                        1000 non-null object
Código SIAFI Município    1000 non-null int64
Nome Município            1000 non-null object
Código Função             1000 non-null int64
Código Subfunção          1000 non-null int64
Código Programa           1000 non-null int64
Código Ação               1000 non-null int64
NIS Favorecido            1000 non-null int64
Nome Favorecido           1000 non-null object
Valor Parcela             1000 non-null float64
Mês Competência           1000 non-null object
Regiao                    1000 non-null object
Estado                    1000 non-null object
Fonte                     1000 non-null object
Finalidade                1000 non-null object
dtypes: float64(1), int64(6), object(8)
memory usage: 125.0+ KB


In [67]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Valor Parcela', 'Mês Competência', 'Regiao',
       'Estado', 'Fonte', 'Finalidade'],
      dtype='object')

In [79]:
for col in df.columns:
    print("'{}'".format(col))

'UF'
'Código SIAFI Município'
'Nome Município'
'Código Função'
'Código Subfunção'
'Código Programa'
'Código Ação'
'NIS Favorecido'
'Nome Favorecido'
'Valor Parcela'
'Mês Competência'
'Regiao'
'Estado'
'Fonte'
'Finalidade'


In [87]:
dia, mes = df['Mês Competência'][10].split('/')

In [107]:
df[df['Nome Município'] == 'PACATUBA']['Código SIAFI Município'].unique()

array([3197, 1495], dtype=int64)

In [140]:
for cod in df[df['Nome Município'] == 'PACATUBA']['Código SIAFI Município'].unique():
    for city in df[df['Código SIAFI Município'] == cod]:
        print(city.split())

['UF']
['Código', 'SIAFI', 'Município']
['Nome', 'Município']
['Código', 'Função']
['Código', 'Subfunção']
['Código', 'Programa']
['Código', 'Ação']
['NIS', 'Favorecido']
['Nome', 'Favorecido']
['Valor', 'Parcela']
['Mês', 'Competência']
['Regiao']
['Estado']
['Fonte']
['Finalidade']
['UF']
['Código', 'SIAFI', 'Município']
['Nome', 'Município']
['Código', 'Função']
['Código', 'Subfunção']
['Código', 'Programa']
['Código', 'Ação']
['NIS', 'Favorecido']
['Nome', 'Favorecido']
['Valor', 'Parcela']
['Mês', 'Competência']
['Regiao']
['Estado']
['Fonte']
['Finalidade']


In [154]:
print("%.2f" % df['Valor Parcela'][0])

131.00
